In [8]:
import requests
import json
import pandas as pd

### Reading the file and create a list from destinations

In [9]:
file = open("dests.txt", "r")
dests_list = file.read().split('\n')

### Use API key and create an empty dictionary

In [10]:
api_key = 'fakeAPIkey'
origin = "Tel Aviv"
dests_dict = {}

### Insert the information to the dictionary

In [11]:
file = open("dests.txt", "r")
for city in dests_list:
    geo_url = 'https://maps.googleapis.com/maps/api/geocode/json?address=%s&key=%s' % (city, api_key)
    dis_url = 'https://maps.googleapis.com/maps/api/distancematrix/json?destinations=%s&origins=%s&key=%s' % (city, origin, api_key)
    try:
        geo_response = requests.get(geo_url)
        dis_response = requests.get(dis_url)
        if not geo_response.status_code == 200 or not dis_response.status_code == 200:
            print('HTTP ERROR', response.status_code)
        else:
            try:
                geo_response_data = geo_response.json()
                dis_response_data = dis_response.json()
            except:
                print("response not in valid JSON format")
            distance = dis_response_data['rows'][0]['elements'][0]['distance']['text']
            duration_sec = int(dis_response_data['rows'][0]['elements'][0]['duration']['value'])
            duration = f"{duration_sec // 3600} hours {duration_sec % 3600 // 60} minutes"
            latitude = str(geo_response_data['results'][0]['geometry']['location']['lat'])
            longitude = str(geo_response_data['results'][0]['geometry']['location']['lng'])
            dests_dict[city]=(distance, duration, latitude, longitude)
    except:
        print("Something went wrong with requests.get")

In [12]:
dests_dict

{'Istanbul': ('1,815 km', '21 hours 2 minutes', '41.0082376', '28.9783589'),
 'Amsterdam': ('4,533 km', '48 hours 3 minutes', '52.3675734', '4.9041389'),
 'Valletta': ('3,793 km', '50 hours 50 minutes', '35.8992375', '14.5140996'),
 'Basel': ('4,093 km', '44 hours 2 minutes', '47.5595986', '7.5885761'),
 'Doha': ('2,164 km',
  '22 hours 38 minutes',
  '25.2854473',
  '51.53103979999999')}

### Create a dataframe that stores the information

In [16]:
df = pd.DataFrame.from_dict(dests_dict, orient='index', columns=["Distance_km", "Duration", "Longitude", "Latitude"])
df.index.name = "Target"
df

,Distance_km,Duration,Longitude,Latitude
Target,,,,
Istanbul,"1,815 km",21 hours 2 minutes,41.0082376,28.9783589
Amsterdam,"4,533 km",48 hours 3 minutes,52.3675734,4.9041389
Valletta,"3,793 km",50 hours 50 minutes,35.8992375,14.5140996
Basel,"4,093 km",44 hours 2 minutes,47.5595986,7.5885761
Doha,"2,164 km",22 hours 38 minutes,25.2854473,51.53103979999999


### Furthest 3 cities from Tel-Aviv

In [14]:
sorted(dests_dict, key = dests_dict.get, reverse = True)[:3]

['Amsterdam', 'Basel', 'Valletta']